# Métodos de aprendizaje automático 2017 laboratorio 1

### Trabajaremos con un conjunto de datos de textos. Para ello los descargamos.
Una descripción del set de datos: http://qwone.com/~jason/20Newsgroups/

In [1]:
from sklearn.datasets import fetch_20newsgroups
train = fetch_20newsgroups(subset='train', categories=['talk.politics.guns', 'soc.religion.christian', 'comp.graphics', 'sci.med'], shuffle=True, random_state=42)
test = fetch_20newsgroups(subset='test', categories=['talk.politics.guns', 'soc.religion.christian', 'comp.graphics', 'sci.med'], shuffle=True, random_state=42)

Mostrar un ejemplo de entrenamiento de cada una de las categorías.

In [2]:
categories=set(['talk.politics.guns', 'soc.religion.christian', 'comp.graphics', 'sci.med'])
cont = 0
while len(categories) != 0 and cont < len(train.data):
    if train.target_names[train.target[cont]] in categories:
        print('************************************************************************')
        print('************************************'+train.target_names[train.target[cont]]+'*****************************')
        print('************************************************************************')
        print train.data[cont]
        categories.remove(train.target_names[train.target[cont]])
    cont += 1

************************************************************************
************************************talk.politics.guns*****************************
************************************************************************
From: tms@cs.umd.edu (Tom Swiss (not Swift, not Suiss, Swiss!))
Subject: Re: Clinton wants National ID card, aka USSR-style "Internal Passport"
Organization: The Reality Liberation Front (pixels to the people!)
Lines: 17

slp9k@cc.usu.edu writes:
>
>	I just want to point out that while I am fully in support of privacy,
>it will be possible soon to have a completely secure ID card, useable in bank
>transactions, medical, etc etc.

     There is no such thing as "completely secure," especially when dealing
with High Technology. It's all a question of cost: what cost are you
willing to bear to protect your information vs. what rewards the "bad guys"
are going to get if they break it. The rewards of breaking such a single ID
system would be high indeed.

Tom Swiss

Revisar el módulo CountVectorizer para extraer caracterísitcas a partir de texto y explicar cómo funciona. Explicar qué es el modelo de bolsa de palabras.

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(train.data)

# CountVectorizer

El módulo CountVectorizer consiste en varias etapas de procesamiento:

Tokenización:Tokeniza los documentos separandolos por espacios en blanco y símbolos de puntuación. A cada diferente token le asigna un id de tipo entero. El patrón para delimitar los tokens se puede pasar por parámetro. 
Conteo:Cuenta las ocurrencias de los tokens en cada documento.
Normalización:normaliza las ocurrencias y penaliza a las palabras que ocurren en la mayoría de los documentos.

Esto luego se lleva a una matriz donde cada fila de la matriz es un documento y cada columna es un token como se explica abajo en la Bolsa de palabras. A este proceso se le llama vectorizar, ya que dado un corpus, lo traduce a un conjunto de características de vectores. 

Este módulo contiene varias opciones, entre las más destacadas se encuentran:
-N gramas: los n-gramas son secuencias de palabras consecutivas. Por ejemplo los bigramas contienen secuencias de dos palabras consecutivas, mientras que los trigramas contienen tres palabras.
-Tokenizador propio: se puede utilizar cualquier tokenizador basado en expresiones regulares.
-Stop words: acepta una lista de palabras que se ignoran para el vector de características. Generalmente estas palabras se ignoran porque se repiten en casi todos los documentos, por lo que no aportan.
-Vocabulario: si se tiene conocimiento sobre el dominio del problema se le puede pasar ya un vocabulario previamente creado.

# Bolsa de palabras

La bolsa de palabras es un modelo para representar un documento o texto, el cual se puede pensar como un conjunto de palabras el cual mantiene la cantidad de ocurrencias de las mismas. Como es un conjunto, esto significa que se pierde el orden de las palabras, lo que termina ocasionando la perdida de la gramática. Es una representación que a priori parece que pierde mucha información, pero en la práctica da muy buenos resultados.

Su representación computacional, puede ser la de una matriz m[i,j] donde i representa un documento y j representa la palabra j y el valor [i,j] representa la cantidad de ocurrencias de la palabra j en el documento i. 
Debido a que un documento generalmente contiene muchas menos palabras que las que se consideran, esta representación de matriz es muy dispersa, por lo que en la práctica se suele utilizar otro tipo de estructuras que aprovechan mejor los recursos.

En función del conocimiento que se tenga sobre la realidad del problema a resolver, se plantean dos grandes formas de modelar mediante bolsa de palabras un documento. Si el conocimiento es mucho y se incurrió en un análisis previo, se puede modelar cada documento con ciertas palabras ya fijadas. En el caso que no haya mucho análisis de la realidad, se pueden utilizar todas las palabras que aparezcan en los documentos. Como regla general, se pueden omitir la lista de palabras denominadas 'stop words' las cuales generalmente no aportan en las decisiones. Dichas 'stop words' pueden ser aprendidas implicitamente con un modelo tf-idf. 

Entrenar un clasificador bayesiano sencillo Multinomial. Explicar cada uno de los parámetros de este clasificador.

In [4]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_counts, train.target)
docs_new = ['God is love', 'OpenGL on the GPU is fast']
X_new_counts = count_vect.transform(docs_new)
predicted = clf.predict(X_new_counts)
for doc, category in zip(docs_new, predicted):
    print('%r => %s' % (doc, train.target_names[category]))

'God is love' => soc.religion.christian
'OpenGL on the GPU is fast' => comp.graphics


El clasificador bayesiano Multinomial tiene 3 parámetros.

Alfa: valor numérico. Este parámetro sirve para el suavizado cuando se aplica máxima verosimilitud para calcular la probabilidad de que alguna caracterísitca (en este caso la palabra) ocurra en determinada clase. El valor por defecto es 1.0, también conocido como el suavizado de Laplace.

fit_prior : booleano. Determina si aprende las probabilidades a priori de las clases a partir de los datos de entrenamiento o si le asigna una probabilidad uniforme a cada clase (ML).

class_prior : arreglo. Arreglo de probabilidades a priori de las clases.

Repetir el proceso pero utilizando el módulo TfidfTransformer en lugar de CountVectorizer. Explicar la medida Tfidf.

In [5]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
clf = MultinomialNB().fit(X_train_tfidf, train.target)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)
predicted = clf.predict(X_new_tfidf)
for doc, category in zip(docs_new, predicted):
    print('%r => %s' % (doc, train.target_names[category]))

'God is love' => soc.religion.christian
'OpenGL on the GPU is fast' => comp.graphics


# TFIDF

En muchos textos, hay palabras que se repiten como artículos, conjunciones, algunos adverbios, pronombres, etc. En general, este tipo de palabra aporta poca información a la hora de clasificar los documentos o en la tarea de recuperación de información. Debido a esto es que se propone el modelo tf-idf, el cual juega con la frecuencia de las palabras en el documento y a su vez penaliza a las palabras que aparecen en muchos documentos.

Con respecto a la frecuencia de las palabras, hay varios modelos, como por ejemplo el de conteo de las palabras como ya vimos en bolsa de palabras. Un modelo muy utilizado es el de frecuencia de logaritmo, el cual le agrega un valor mas alto de frecuencia a las palabras que aparecen mas en el documento, pero dicho aumento no es lineal con la cantidad de ocurrencias, sino que es logaritmico. Otro posible modelo puede tener en cuenta la cantidad de palabras en el documento para determinar su frecuencia.

La frecuencia inversa del documento es una medida de cuanta información provee la palabra, es decir si el término es común o es raro a través de los documentos. La idea detrás es que los términos que aparecen pocas veces aportan mucha más información que los que aparecen mucho en todos los documentos. Para calcular este valor, primero se calcula la cantidad de documentos en los que aparece el término, luego se divide la cantidad total de documentos sobre este valor y finalmente se le aplica el logaritmo al resultado. Ese resultado es la frecuencia inversa del documento para un término.

Finalmente cabe destacar que la medida tf-idf se calcula como el producto tf * idf.


Incluya los pasos de extracción de características y el clasificador dentro de un pipeline.

In [6]:
from sklearn.pipeline import Pipeline
pipe = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf', MultinomialNB()),
 ])

El uso del pipeline es por simplicidad y además porque es más facil a la hora de probar con diferentes valores en los hiper parámetros.

Incluir las métricas de precision, recall y fscore para cada cada una de las clases y la matriz de confusión probando contra el conjunto de testing.

In [7]:
import numpy as np
from sklearn import metrics

pipe.fit(train.data, train.target)
docs_test = test.data
predicted = pipe.predict(docs_test)

from sklearn import metrics
print(metrics.classification_report(test.target, predicted, target_names=test.target_names))
metrics.confusion_matrix(test.target, predicted)

                        precision    recall  f1-score   support

         comp.graphics       0.96      0.89      0.92       389
               sci.med       0.99      0.81      0.89       396
soc.religion.christian       0.80      0.99      0.89       398
    talk.politics.guns       0.94      0.96      0.95       364

           avg / total       0.92      0.91      0.91      1547



array([[345,   2,  29,  13],
       [  9, 320,  57,  10],
       [  2,   1, 395,   0],
       [  2,   1,  11, 350]])

A partir de las métricas se puede ver un muy buen resultado en todas las clases y en todas las medidas de performance. 
En base a la matriz de confusión, se puede ver que la predicción de la religión es la que se podría mejorar, ya que es la columna que presenta los valores más altos.

Incluya una GridSearch para ver cuales son los mejores hiper-parámetros del modelo

In [8]:
from sklearn.model_selection import GridSearchCV
parameters = {'vect__ngram_range': [(1, 1), (1, 2)],
              'vect__stop_words' : ('english', None),
              'vect__binary' : (True,False),
              'tfidf__norm' : ('l1','l2',None),
              'tfidf__use_idf' : (True,False),
              'tfidf__smooth_idf' : (True,False),
}

Los hiper-parámetros del clasificador MultinomialNB fueron explicados anteriormente. 
En cuanto al CountVectorizer, se eligieron tres parámetros:

ngram_range: Considera dos posibilidades, unigrama y bigramas. Los ngramas son un modelo de lenguaje. El caso del unigrama es bien sencillo, simplemente cuenta la cantidad de ocurrencias de la palabra. Mientras que en el caso de los bigramas además cuenta las ocurrencias de a dos palabras juntas.

stop_words: Las stop_words como ya se explicó, son palabras del idioma que generalmente no aportan contenido para la clasificación, ya que son palabras que ocurren en casi todos los documentos. Los valores posibles para esta ejecución son de ignorarlas o de considerarlas.

binary: El modelo binario es el más simple de todos. No cuenta las ocurrencias de las caracerísticas en el modelo, sino que pone 1 cuando aparece la característica en el documento y 0 cuando no aparece. Basicamente cuenta si aparece la palabra o no en el documento, no cuantas veces aparece.

Para el TfidfTransformer, se eligieron tres parámetros:

norm: refiere al tipo de función a computar para el cálculo de la frecuencia del término. Puede ser la función logarítmica, la función coseno o simplemente el conteo.

use_idf: Si habilitar o deshabilitar idf.

smooth_idf: Esto es para el suavizado del idf.

Exlique cada uno de los parámetros de cada uno de los pasos del pipeline y pruebe con varios tipos de clasificadores de los vistos en el curso. ¿Con qué configuración de parámetros obtuvo mejores resultados? Aplique selección de atributos y valore los resultados obtenidos. Realice una prueba final contra el conjunto de pruebas.

In [12]:
nb_clf = GridSearchCV(pipe, parameters, n_jobs=-1).fit(train.data, train.target)
from sklearn.neighbors import KNeighborsClassifier
pipe = Pipeline([('vect', CountVectorizer()),
                  ('tfidf', TfidfTransformer()),
                  ('clf', KNeighborsClassifier()),
])

knn_clf = GridSearchCV(pipe, parameters, n_jobs=-1).fit(train.data, train.target)

from sklearn.tree import DecisionTreeClassifier

pipe = Pipeline([('vect', CountVectorizer()),
                  ('tfidf', TfidfTransformer()),
                  ('clf', DecisionTreeClassifier()),
])

id3_clf = GridSearchCV(pipe, parameters, n_jobs=-1).fit(train.data, train.target)

print 'Accuracy:'
print 'Naive Bayes:' + str(nb_clf.best_score_)
print 'KNN:' + str(knn_clf.best_score_)
print 'ID3:' + str(id3_clf.best_score_)

Accracy:
Naive Bayes:0.985794231597
KNN:0.949203616014
ID3:0.854928971158


Accuracy:
Naive Bayes:0.985794231597
KNN:0.949203616014
ID3:0.854928971158

Considerando la medida Accuracy como medida de performance para evaluar los modelos, se desprende que el mejor clasificador fue Naive Bayes con los siguientes hiper-parámetros del modelo:
{'vect__ngram_range': (1, 2), 'tfidf__smooth_idf': True, 'tfidf__norm': None, 'vect__binary': False, 'tfidf__use_idf': True, 'vect__stop_words': None}


In [17]:
from sklearn import feature_selection


count_feats = range(100,200000,1000)
parameters = {'feat__k' : count_feats}
pipe = Pipeline([('vect', CountVectorizer(ngram_range=(1,2))),
                 ('feat', feature_selection.SelectKBest(feature_selection.chi2)),
                  ('tfidf', TfidfTransformer()),
                  ('clf', MultinomialNB()),
])
nb_feat_select_clf = GridSearchCV(pipe, parameters, n_jobs=-1).fit(train.data, train.target)    

In [19]:
print nb_feat_select_clf.best_score_
print nb_feat_select_clf.best_params_

0.964700817908
{'feat__k': 10100}


Utilizando selección de atributos y con un gap de 1000, vemos que la mejor performance es cuando tiene 10100 atributos. 
0.964700817908
{'feat__k': 10100}
Aún así, funciona mejor con la totalidad de los atributos.


In [18]:
pipe = Pipeline([('vect', CountVectorizer(ngram_range=(1,2))),
                  ('tfidf', TfidfTransformer()),
                  ('clf', MultinomialNB())
])

pipe.fit(train.data, train.target)
predicted = pipe.predict(test.data)
np.mean(predicted == test.target)

0.89851325145442795

Si bien la accuracy utilizando cross-validation (el por defecto 3) en GridSearch fue de 98.5%, para el conjunto de testeo es de casi un 10% menos. Se podría haber aumentado el k en k-folding a 10 tal vez y haber hecho pruebas con eso, pero por temas de tiempo, se eligió el por defecto. Aún así, la prueba de accuracy utilizando cross-validation es más representativa que probando contra el conjunto de testeo ya que el cross-validation no sufre de 'high-variance estimate' ya que por defecto GridSearch se encarga de crear los los conjuntos k bien distribuidos en cuanto a sus documentos.

Repita el ejercicio utilizando HashingVectorizer para extraer características. Explique las ventajas de este método de extracción de caraterísticas frente a CountVectorizer.

In [27]:
from sklearn.feature_extraction.text import HashingVectorizer

parameters = {'vect__n_features' : count_feats}
pipe = Pipeline([('vect', HashingVectorizer(ngram_range=(1,2), non_negative=True)),
                  ('clf', MultinomialNB()),
])
nb_hashing_clf = GridSearchCV(pipe, parameters, n_jobs=-1).fit(train.data, train.target)  

predicted = nb_hashing_clf.predict(test.data)
print nb_hashing_clf.best_score_
print nb_hashing_clf.best_params_
print np.mean(predicted == test.target)

0.85972850678733037

0.907447266466
{'vect__n_features': 11100}
0.86102133161

Vemos que utilizando CountVectorizer +TfIdfTransformer anduvo mejor que utilizando HashingVectorizer. El primero tuvo un 8% mas de accuracy sobre el cross validation y un 4% sobre el cuerpo de testeo. A partir de la cantidad de características elegidas, podemos resaltar que esa cantidad es muy parecida a la que devolvió el método SelectKBest.

Las principales ventajas están en la forma de almacenamiento. Utiliza una función de Hash para pasar los términos a un índice, por lo que ahorra espacio de memoria al no tener que guardar un diccionario. También viene con un parámetro opcional para elegir la cantidad de features a representar. Por otro lado, la matriz que utiliza es dispersa, por lo que no guarda los 0's en los documentos que no tienen esa característica. 